Install Packages

In [ ]:
%pip install langchain -U
%pip install tiktoken
%pip install openai -U 
%pip install python-dotenv
%pip install context-python -U

In [43]:
import os
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
import pinecone

In [44]:
load_dotenv()

EMBEDDING_MODEL: str = os.environ["EMBEDDING_MODEL"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_API_ENV = os.environ["PINECONE_API_ENV"]
CONTEXT_API_TOKEN = os.environ["CONTEXT_API_TOKEN"]

In [45]:
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

In [32]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "alzh" # put in the name of your pinecone index here

In [46]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFDirectoryLoader("./docs/")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Create a new index
docsearch = Pinecone.from_texts([t.page_content for t in texts], metadatas=[t.metadata for t in texts], embedding=embeddings, index_name=index_name)

In [47]:
# Map to an existing index
index = pinecone.Index(index_name)

vector_store = Pinecone(index, embeddings.embed_query, text_key="text")

In [22]:
# Using davinci completion using QA chain

# from langchain.chains.question_answering import load_qa_chain
# from langchain.llms import OpenAI
# from langchain.callbacks import get_openai_callback

# with get_openai_callback() as cb:
#     chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")

#     print(chain.run(input_documents=docs, question=query))

#     print(cb)

In [51]:
# Using GPT-3.5 with a conversational memory
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.callbacks import get_openai_callback
from langchain.callbacks import ContextCallbackHandler

context_callback = ContextCallbackHandler(CONTEXT_API_TOKEN)

with get_openai_callback() as cb:
    llm = ChatOpenAI(
        openai_api_key=os.environ["OPENAI_API_KEY"],
        model_name='gpt-3.5-turbo',
        temperature=0.0,
        callbacks=[context_callback]
    )
    
    qa = RetrievalQAWithSourcesChain.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever()
    )

    query = "How does a blood test for Alzheimer's work?"
    
    result = qa({"question": query})

    print(result["answer"])
    print("Sources:")
    print(result["sources"])
    print()
    print(cb)

The process of a blood test for Alzheimer's disease involves the measurement of specific biomarkers in the blood that are associated with the disease. These biomarkers can include amyloid beta (Aβ) proteins, tau proteins, and neurofilament light chain (NfL). The levels of these biomarkers in the blood can provide information about the presence and progression of Alzheimer's disease. The use of blood biomarkers in Alzheimer's disease research and clinical trials is a rapidly evolving field and shows promise for improving diagnosis and monitoring treatment effects. However, further research and validation are needed before blood tests for Alzheimer's disease can be widely implemented in clinical practice. 


Sources:
1. Hansson O, Edelmayer RM, Boxer AL, et al. The Alzheimer’s Association appropriate use recommendations for blood biomarkers in Alzheimer’s disease. Alzheimers Dement. 2022;18(12):2669-2686. doi:10.1002/alz.12756
2. Schindler SE, Li Y, Li M, et al. Using Alzheimer’s disease